In [9]:
import random
import datetime
import numpy as np
import h5py
from skimage.filters import threshold_otsu
from skimage import io
from matplotlib import pyplot as plt
import math
import tifffile
import os
import sys
import shutil
from skimage.draw import polygon as ski_polygon
import json

import openslide
from preprocessing.datamodel import SlideManager
from preprocessing.processing import split_negative_slide, split_positive_slide, create_tumor_mask, rgb2gray
from preprocessing.util import TileMap

import calculate_performance as calc

%matplotlib inline

In [10]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from sklearn.metrics import roc_curve, auc
from scipy.special import softmax #e to the x and divide by sum
import matplotlib.pyplot as plt
import time
from datetime import date
import os
import sys
import copy
from tqdm import tqdm
import shutil

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.6.0
Torchvision Version:  0.7.0


In [11]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [12]:
def image_show(image, mymax, nrows=1, ncols=1, cmap='gray',size = 8):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(size, size*image.shape[0]/image.shape[1]))
    try:
        mymax
        ax.imshow(image, vmax = mymax, cmap='gray')
    except NameError:
        ax.imshow(image, cmap='gray')
    ax.axis('off')
    return fig, ax

In [13]:
def copy_test_case_patches(path, test_case):  
    if os.path.exists(path + '//test//') == False:
        os.mkdir(path + '//test//')
        
    # Make folder to move all test patches to
    if os.path.exists(path + '//test//' + str(test_case)) == False:
        os.mkdir(path + '//test//' + str(test_case) + '//')
        os.mkdir(path + '//test//' + str(test_case) + '//0//')
        os.mkdir(path + '//test//' + str(test_case) + '//1//')
    print('created patch-based test folder in path: ' + str(path) + '//test//' + str(test_case) + '//')

    ## Copy test case 0 patches to the new folder
    for file in os.listdir(path + '//all//0//'):
        if file.startswith(test_case):
            source = path + '//all//0//' + file
            dest = path + '//test//' + str(test_case) + '//0//' + file
            shutil.copy(source, dest) #move to designated path

    ## Copy test case 1 patches to the new folder
    for file in os.listdir(path + '//all//1//'):
        if file.startswith(test_case):
            source = path + '//all//1//' + file
            dest = path + '//test//' + str(test_case) + '//1//' + file
            shutil.copy(source, dest) #move to designated path

In [14]:
DIR = ''

mgr = SlideManager(cam16_dir=DIR)

slides_met = mgr.met_slides

N_met = len(slides_met)

slides_negative = mgr.negative_slides

N_negative = len(slides_negative)

level = 0

tile_size = 512 #must be the same as used for training

poi = 0.50 #must use the same poi we used to seperate tisse from background

overlap = tile_size // 2 #increasing overlap will put patches closer together

input_size = tile_size

n_folds = 15 

In [15]:
data_dir = ".//patches//test//"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Test all models, save all ROC data

In [32]:
prediction_folder_path = 'predictions//Kfold//'
patches_path = 'patches//'
auc_all = []
for fold in range(n_folds):
    for file in os.listdir():
        if file.startswith('resnet_Kfold_' + str(fold) + '_') and file.endswith('.pt'): ##
            
            ## Define test cases for this model
            test_case = []
            cases = file.split('(')[1].split(')')[0] ## Grab names of cases for this model. Format 'case', 'case' 
            test_case.append(cases.split(',')[0][1:-1]) ## Grab first case. Cut off ' ' at the beg/end of name
            test_case.append(cases.split(',')[1][2:-1]) ## Grab second case. Cut off space at beg, and ' ' at the beg/end of name 

            ## Load model
            model_name = file
            model = torch.load(model_name)
            model.eval()

            try:
                all_outputs
                del all_outputs
            except NameError:
                print('initializing variables')
            try:
                all_labels
                del all_labels
            except NameError:
                pass

            for ii in range(len(test_case)):                      
                dataset_name = test_case[ii]
                
                if os.path.exists(patches_path + '//test//' + str(test_case[ii])) == False: ## If test patches do NOT exist,
                    copy_test_case_patches(patches_path, dataset_name) ## then copy test patches to folder
                    print('copying test patches for ' + str(dataset_name))
                
                print('Making predictions for ' + str(dataset_name))
                print('Model name = ' + str(model_name))

                running_loss = 0.0
                running_corrects = 0
                n_classes = 2

                data_transforms = {
                dataset_name: transforms.Compose([
                transforms.Resize(input_size),
                transforms.CenterCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

                # Create dataloader
                image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in [dataset_name]}
                dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in [dataset_name]}

                # Detect if we have a GPU available
                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
                running_corrects = 0
                # Iterate over data.
                for inputs, labels in dataloaders_dict[dataset_name]: #get inputs. data is list of [inputs, labels]
                    out = torchvision.utils.make_grid(inputs) #for visualizing inputs
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    outputs = model(inputs) #output is not normalized
                    #criterion function normalizes the outputs into probabilities with softmax
                    loss = criterion(outputs, labels) #loss being nn.CrossEntropy
                    _, preds = torch.max(outputs, 1) #prediction is given to maximum probability. i.e. threshold = 0.50

                    labels_cpu = labels.cpu().detach().numpy()
                    preds_cpu = preds.cpu().detach().numpy()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data) #Number of corrects in each batch

                    ## Store ALL labels and outputs for plot
                    try:
                        all_outputs
                        all_outputs = np.append(all_outputs, outputs.cpu().detach().numpy(), axis = 0) 
                        all_labels = np.append(all_labels, labels.cpu().detach().numpy(), axis = 0)
                    except NameError:
                        all_outputs = outputs.cpu().detach().numpy() #initial element of all_outputs
                        all_labels = labels.cpu().detach().numpy() #initial element of all_outputs

            ## Get ROC curve for both test cases from this model
            fpr = dict()
            tpr = dict()
            roc_auc = dict()
            outputs_normalized = softmax(all_outputs, axis = 1)

            #create y_score (matrix of labels, 2 columns, 1 in column for positive class)
            y_score = np.zeros((len(all_labels),2))
            for col in range(len(all_labels)):
                y_score[col,all_labels[col]] = 1 #if labels_cpu[col] is 1, col 1 = 1
            for i in range(2):
                fpr[i], tpr[i], _ = roc_curve(y_score[:,i], outputs_normalized[:,i])
                roc_auc[i] = auc(fpr[i],tpr[i])
                
            ## Save records of this test 
            np.save(prediction_folder_path + '//ROC//' + str(fold) + '_fpr.npy', fpr[1])
            np.save(prediction_folder_path + '//ROC//' + str(fold) + '_tpr.npy', tpr[1])
            np.save(prediction_folder_path + '//ROC//outputs_labels//' + str(fold) + '_all_labels.npy', all_labels)
            np.save(prediction_folder_path + '//ROC//outputs_labels//' + str(fold) + '_all_outputs.npy', all_outputs)
        

Making predictions for 08159A1-a
Model name = resnet_Kfold_0_('08159A1-a', '7115B-a').pt
Making predictions for 7115B-a
Model name = resnet_Kfold_0_('08159A1-a', '7115B-a').pt
